In [1]:
import warnings
warnings.simplefilter("ignore")

import datetime, os
from functools import partial
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.data import Dataset
from tensorflow import keras
import keras_tuner as kt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

In [2]:
# load the TensorBoard notebook extension
%load_ext tensorboard

In [3]:
# Set paths

DATA_PATH = 'Reservoir_Project/Data'
HP_TUNING_PATH = 'Reservoir_Project/Hyperparameter_Tuning'

In [4]:
stationary_basin_inflow = pd.read_excel(f'{DATA_PATH}/Custom/stationary_basin_inflow.xlsx', index_col=0)

#### Scale data for training

In [5]:
"""
Normalizing data

Maximum absolute scaling rescales each feature between -1 and 1 by dividing every observation 
by its maximum absolute value.
"""

scaler = MinMaxScaler(feature_range=(-1, 1))
normalized_df = scaler.fit_transform(stationary_basin_inflow)

normalized_basin_inflow = pd.DataFrame(normalized_df, columns=stationary_basin_inflow.columns) 

normalized_basin_inflow.head()

,INFLOW,NFD_MEAN_FLOW,OXB_RIVER_STAGE,OXB_RIVER_DISCHARGE,CBR_RIVER_STAGE,CBR_RIVER_DISCHARGE,ADR_PRECIP_ACC,ADR_PRECIP_INCR,ADR_TEMP_AVG,ADR_TEMP_MAX,...,FRN_SNOW_DEPTH,FRN_SNOW_WATER_CONTENT,FRN_TEMP_AVG,FRN_TEMP_MAX,FRN_TEMP_MIN,PFH_PRECIP_ACC,PFH_PRECIP_INCR,PFH_TEMP_AVG,PFH_TEMP_MAX,PFH_TEMP_MIN
0,0.046224,0.051876,0.237069,0.063465,-0.401142,0.052915,-0.193228,0.317948,-0.136617,-0.445268,...,0.056802,-0.047357,-0.467484,-0.513988,-0.433957,-0.015267,0.226186,-0.175380,-0.606621,-0.203656
1,0.046656,0.057364,0.244207,0.064644,-0.200977,0.026400,-0.187313,0.353458,-0.248239,-0.443910,...,0.061424,-0.045370,-0.342164,-0.361327,-0.314145,-0.010052,0.300835,-0.168862,-0.689554,-0.172624
2,0.040509,0.060470,0.248504,0.064578,-0.182704,0.000397,-0.184687,0.222780,-0.263400,-0.327549,...,0.065748,-0.043497,-0.183765,-0.177308,-0.160303,-0.004875,0.299022,-0.214953,-0.582226,-0.206942
3,0.036760,0.061464,0.204183,0.046449,-0.152004,-0.003311,-0.195223,-0.324864,-0.287873,-0.270221,...,0.064547,-0.043564,-0.118437,-0.073134,-0.118773,-0.015040,-0.428473,-0.223444,-0.454890,-0.280173
4,-0.002739,0.061806,0.154703,0.031689,-0.165334,-0.016063,-0.205048,-0.399602,-0.225255,-0.180684,...,0.075019,-0.043065,-0.070014,0.009671,-0.089780,-0.023395,-0.465585,-0.139686,-0.279272,-0.303446


#### Split data

In [6]:
# 70/20/10 split for training, validation, and test sets

df_size = len(normalized_basin_inflow)

basin_inflow_train = normalized_basin_inflow[0:int(df_size * 0.7)] # 70% 

basin_inflow_validation = normalized_basin_inflow[int(df_size * 0.7):int(df_size * 0.9)] # next 20%
basin_inflow_validation.reset_index(drop=True, inplace=True)

basin_inflow_test = normalized_basin_inflow[int(df_size * 0.9):] # last 10%
basin_inflow_test.reset_index(drop=True, inplace=True)

In [7]:
basin_inflow_test.tail()

,INFLOW,NFD_MEAN_FLOW,OXB_RIVER_STAGE,OXB_RIVER_DISCHARGE,CBR_RIVER_STAGE,CBR_RIVER_DISCHARGE,ADR_PRECIP_ACC,ADR_PRECIP_INCR,ADR_TEMP_AVG,ADR_TEMP_MAX,...,FRN_SNOW_DEPTH,FRN_SNOW_WATER_CONTENT,FRN_TEMP_AVG,FRN_TEMP_MAX,FRN_TEMP_MIN,PFH_PRECIP_ACC,PFH_PRECIP_INCR,PFH_TEMP_AVG,PFH_TEMP_MAX,PFH_TEMP_MIN
518,-0.057453,-0.069672,-0.048471,-0.055036,0.110833,0.030613,-0.315073,-0.167686,-0.028035,0.076778,...,0.161965,0.002282,0.176226,0.273295,0.082711,-0.172650,-0.067776,0.019043,-0.024976,-0.232790
519,-0.046580,-0.049674,-0.034969,-0.046170,0.137346,0.046753,-0.321772,-0.149572,-0.019661,0.083031,...,0.148337,-0.002336,0.181140,0.291265,0.079625,-0.175266,-0.057266,0.020266,-0.025389,-0.204498
520,-0.035821,-0.041221,-0.028311,-0.042407,0.175412,0.066402,-0.326896,-0.087380,0.004036,0.076013,...,0.135179,-0.006897,0.196598,0.305683,0.088279,-0.177496,-0.039123,0.050523,-0.028502,-0.145959
521,0.003645,0.025585,-0.011379,-0.028597,0.218526,0.094591,-0.326242,0.141349,0.038101,0.072664,...,0.123689,-0.009845,0.230065,0.328739,0.128114,-0.173514,0.256248,0.100663,-0.023233,-0.068289
522,0.207759,0.196131,0.012426,-0.008109,0.298208,0.170347,-0.317809,0.449297,0.058071,0.079183,...,0.112940,-0.012843,0.250512,0.350308,0.142296,-0.164637,0.488933,0.133012,-0.015576,-0.018538


In [8]:
basin_inflow_train.head()

,INFLOW,NFD_MEAN_FLOW,OXB_RIVER_STAGE,OXB_RIVER_DISCHARGE,CBR_RIVER_STAGE,CBR_RIVER_DISCHARGE,ADR_PRECIP_ACC,ADR_PRECIP_INCR,ADR_TEMP_AVG,ADR_TEMP_MAX,...,FRN_SNOW_DEPTH,FRN_SNOW_WATER_CONTENT,FRN_TEMP_AVG,FRN_TEMP_MAX,FRN_TEMP_MIN,PFH_PRECIP_ACC,PFH_PRECIP_INCR,PFH_TEMP_AVG,PFH_TEMP_MAX,PFH_TEMP_MIN
0,0.046224,0.051876,0.237069,0.063465,-0.401142,0.052915,-0.193228,0.317948,-0.136617,-0.445268,...,0.056802,-0.047357,-0.467484,-0.513988,-0.433957,-0.015267,0.226186,-0.175380,-0.606621,-0.203656
1,0.046656,0.057364,0.244207,0.064644,-0.200977,0.026400,-0.187313,0.353458,-0.248239,-0.443910,...,0.061424,-0.045370,-0.342164,-0.361327,-0.314145,-0.010052,0.300835,-0.168862,-0.689554,-0.172624
2,0.040509,0.060470,0.248504,0.064578,-0.182704,0.000397,-0.184687,0.222780,-0.263400,-0.327549,...,0.065748,-0.043497,-0.183765,-0.177308,-0.160303,-0.004875,0.299022,-0.214953,-0.582226,-0.206942
3,0.036760,0.061464,0.204183,0.046449,-0.152004,-0.003311,-0.195223,-0.324864,-0.287873,-0.270221,...,0.064547,-0.043564,-0.118437,-0.073134,-0.118773,-0.015040,-0.428473,-0.223444,-0.454890,-0.280173
4,-0.002739,0.061806,0.154703,0.031689,-0.165334,-0.016063,-0.205048,-0.399602,-0.225255,-0.180684,...,0.075019,-0.043065,-0.070014,0.009671,-0.089780,-0.023395,-0.465585,-0.139686,-0.279272,-0.303446


### Data windowing

In [9]:
# TensorFlow utility class for producing data windows from time series data

class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df, val_df, test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df
    
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}
    
    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

In [10]:
"""
Window
- Given 60 days of history predict 30 days into the future. Why? A season is about 90 days in a CA WY
- Window size: 90
"""

window = WindowGenerator(input_width=60, label_width=1, shift=30,
                     train_df=basin_inflow_train, val_df=basin_inflow_validation, 
                     test_df=basin_inflow_test, label_columns=['INFLOW'])

window

In [11]:
# create a window of inputs and labels

def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # set shapes after slicing
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [12]:
# create a dataset of sliding windows over a time series dataframe

def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  # (input_window, label_window) pairs 
  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [13]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test

In [14]:
# each element is an (inputs, label) pair
window.train.element_spec

(TensorSpec(shape=(None, 60, 39), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 1, 1), dtype=tf.float32, name=None))

In [15]:
# example batch
for inputs, labels in window.train.take(1):
  print(f'Inputs shape (batch, time, features): {inputs.shape}')
  print(f'Labels shape (batch, time, features): {labels.shape}')

Inputs shape (batch, time, features): (32, 60, 39)
Labels shape (batch, time, features): (32, 1, 1)


### Modeling

In [16]:
val_performance = {}
test_performance = {}

#### Create baseline

In [17]:
# tensorflow baseline utility class for data windowing

class Baseline(tf.keras.Model):
  def __init__(self, label_index=None):
    super().__init__()
    self.label_index = label_index

  def call(self, inputs):
    if self.label_index is None:
      return inputs
    result = inputs[:, :, self.label_index]
    return result[:, :, tf.newaxis]

In [18]:
baseline = Baseline(label_index=window.column_indices['INFLOW'])

baseline.compile(loss=tf.keras.losses.MeanAbsoluteError(),
                 metrics=[tf.keras.metrics.MeanAbsoluteError()])

val_performance['Baseline'] = baseline.evaluate(window.val)
test_performance['Baseline'] = baseline.evaluate(window.test, verbose=0)

30/30 [==============================] - 0s 490us/step - loss: 0.0870 - mean_absolute_error: 0.0870


#### LSTM

In [19]:
MAX_EPOCHS = 10

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.compile(loss=tf.keras.losses.MeanAbsoluteError(),
                optimizer=tf.keras.optimizers.legacy.Adam())

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping, tensorboard_callback])
  return history

LSTM 1

In [20]:
"""
Inputs shape (batch, time, features): (32, 60, 39) - 32 batch size, 60 time steps, 39 features
"""

lstm_v1 = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True, input_shape=[None, 39]),
    tf.keras.layers.Dense(units=1)
])

In [21]:
history_lstm_v1 = compile_and_fit(lstm_v1, window)

Epoch 1/10
112/112 [==============================] - 2s 9ms/step - loss: 0.0857 - val_loss: 0.0776
Epoch 2/10
112/112 [==============================] - 1s 7ms/step - loss: 0.0622 - val_loss: 0.0880
Epoch 3/10
112/112 [==============================] - 1s 7ms/step - loss: 0.0567 - val_loss: 0.0960


In [22]:
# %tensorboard --logdir logs

In [23]:
val_performance['LSTM1'] = lstm_v1.evaluate(window.val)
test_performance['LSTM1'] = lstm_v1.evaluate(window.test, verbose=0)

print('LSTM 1 Validation performance: ', val_performance['LSTM1'])
print('LSTM 1 Test performance: ', test_performance['LSTM1'])

30/30 [==============================] - 0s 2ms/step - loss: 0.0960
LSTM 1 Validation performance:  0.09596401453018188
LSTM 1 Test performance:  0.04411937668919563


LSTM 2

In [24]:
lstm_v2 = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True, input_shape=[None, 39]),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(units=1)
])

In [25]:
history_lstm_v2 = compile_and_fit(lstm_v2, window)

Epoch 1/10
112/112 [==============================] - 4s 18ms/step - loss: 0.0680 - val_loss: 0.0702
Epoch 2/10
112/112 [==============================] - 2s 14ms/step - loss: 0.0489 - val_loss: 0.0840
Epoch 3/10
112/112 [==============================] - 2s 14ms/step - loss: 0.0413 - val_loss: 0.0970


In [26]:
val_performance['LSTM2'] = lstm_v2.evaluate(window.val)
test_performance['LSTM2'] = lstm_v2.evaluate(window.test, verbose=0)

print('LSTM 2 Validation performance: ', val_performance['LSTM2'])
print('LSTM 2 Test performance: ', test_performance['LSTM2'])

30/30 [==============================] - 0s 3ms/step - loss: 0.0970
LSTM 2 Validation performance:  0.09702549874782562
LSTM 2 Test performance:  0.05021598935127258


LSTM 3

In [27]:
RegularizedLSTM = partial(tf.keras.layers.LSTM,
                           dropout=0.01,
                           recurrent_dropout=0.01,
                           kernel_regularizer=tf.keras.regularizers.l2(0.05))

In [28]:
lstm_v3 = tf.keras.models.Sequential([
    RegularizedLSTM(32, return_sequences=True, input_shape=[None, 39]),
    RegularizedLSTM(32),
    tf.keras.layers.Dense(units=1)
])

In [29]:
history_lstm_v3 = compile_and_fit(lstm_v3, window)

Epoch 1/10
112/112 [==============================] - 7s 34ms/step - loss: 2.6739 - val_loss: 0.8742
Epoch 2/10
112/112 [==============================] - 4s 34ms/step - loss: 0.3962 - val_loss: 0.1390
Epoch 3/10
112/112 [==============================] - 4s 34ms/step - loss: 0.0955 - val_loss: 0.0667
Epoch 4/10
112/112 [==============================] - 4s 35ms/step - loss: 0.0686 - val_loss: 0.0715
Epoch 5/10
112/112 [==============================] - 4s 35ms/step - loss: 0.0659 - val_loss: 0.0621
Epoch 6/10
112/112 [==============================] - 4s 35ms/step - loss: 0.0666 - val_loss: 0.0670
Epoch 7/10
112/112 [==============================] - 4s 35ms/step - loss: 0.0651 - val_loss: 0.0701


In [30]:
val_performance['LSTM3'] = lstm_v3.evaluate(window.val)
test_performance['LSTM3'] = lstm_v3.evaluate(window.test, verbose=0)

print('LSTM 3 Validation performance: ', val_performance['LSTM3'])
print('LSTM 3 Test performance: ', test_performance['LSTM3'])

30/30 [==============================] - 0s 4ms/step - loss: 0.0701
LSTM 3 Validation performance:  0.07005796581506729
LSTM 3 Test performance:  0.045485720038414


#### Hyperparameter tuning

In [31]:
def build_model(hp):
  lstm = keras.Sequential()

  ### Tuning hyperparameters ### 

  # Number of lstm layer units
  hp_units = hp.Int('units', min_value=32, max_value=128, step=32)

  # Dropout rate applied to input values
  hp_dropout = hp.Choice("dropout", [0.2, 0.3, 0.4, 0.5])

  # Recurrent dropout rate applied to hidden cell states between time steps
  hp_recurrent_dropout = hp.Choice("recurrent_dropout", [0.2, 0.3, 0.4, 0.5])
  
  # L2 regularization 
  hp_l2_reg = hp.Choice("l2", [0.001, 0.01, 0.02, 0.05])

  # Optimizer learning rate
  hp_learning_rate = hp.Choice('learning_rate', values=[0.01, 0.001, 0.0001])

  ### Layers ### 
  
  lstm.add(tf.keras.layers.LSTM(units=hp_units, dropout=hp_dropout, recurrent_dropout=hp_recurrent_dropout, 
                  kernel_regularizer=tf.keras.regularizers.l2(l2=hp_l2_reg),
                  return_sequences=True, input_shape=[None, 39]))
    
  lstm.add(tf.keras.layers.LSTM(units=hp_units, dropout=hp_dropout, recurrent_dropout=hp_recurrent_dropout, 
                  kernel_regularizer=tf.keras.regularizers.l2(l2=hp_l2_reg)))
    
  lstm.add(keras.layers.Dense(1))    
    
  ### Compile ### 
           
  lstm.compile(loss=tf.keras.losses.MeanAbsoluteError(),
               optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=hp_learning_rate),
               metrics=['mean_absolute_error'])

  return lstm

In [ ]:
tuner = kt.Hyperband(build_model,
                     objective='val_mean_absolute_error',
                     max_epochs=10,
                     factor=3,
                     directory=HP_TUNING_PATH,
                     project_name='reservoir_model_hp_tuning_v3')

In [33]:
MAX_EPOCHS = 10
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min')

In [34]:
# args for search are those used for model fit
tuner.search(window.train, epochs=MAX_EPOCHS, validation_data=window.val, callbacks=[early_stopping])

In [35]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete and below are the optimal values:

- Units: {best_hps.get('units')}
- Dropout: {best_hps.get('dropout')}
- Recurrent dropout: {best_hps.get('recurrent_dropout')}
- L2: {best_hps.get('l2')}
- Learning rate: {best_hps.get('learning_rate')}
""")


The hyperparameter search is complete and below are the optimal values:

- Units: 32
- Dropout: 0.4
- Recurrent dropout: 0.3
- L2: 0.02
- Learning rate: 0.01



#### Train

In [36]:
# first find the optimal number of training epochs

MAX_EPOCHS = 10

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                patience=2,
                                                mode='min')

model = tuner.hypermodel.build(best_hps)
history = model.fit(window.train, epochs=MAX_EPOCHS, validation_data=window.val, callbacks=[early_stopping])

val_mae_per_epoch = history.history['val_mean_absolute_error']
best_epoch = val_mae_per_epoch.index(min(val_mae_per_epoch)) + 1 # find epoch of lowest validation MAE 
print('Best epoch: %d' % (best_epoch,))

Epoch 1/10
112/112 [==============================] - 6s 34ms/step - loss: 0.2255 - mean_absolute_error: 0.0728 - val_loss: 0.0615 - val_mean_absolute_error: 0.0559
Epoch 2/10
112/112 [==============================] - 4s 34ms/step - loss: 0.0726 - mean_absolute_error: 0.0666 - val_loss: 0.0605 - val_mean_absolute_error: 0.0561
Epoch 3/10
112/112 [==============================] - 4s 34ms/step - loss: 0.0734 - mean_absolute_error: 0.0670 - val_loss: 0.0592 - val_mean_absolute_error: 0.0541
Epoch 4/10
112/112 [==============================] - 4s 34ms/step - loss: 0.0712 - mean_absolute_error: 0.0657 - val_loss: 0.0624 - val_mean_absolute_error: 0.0576
Epoch 5/10
112/112 [==============================] - 4s 35ms/step - loss: 0.0711 - mean_absolute_error: 0.0657 - val_loss: 0.0575 - val_mean_absolute_error: 0.0515
Epoch 6/10
112/112 [==============================] - 4s 34ms/step - loss: 0.0706 - mean_absolute_error: 0.0654 - val_loss: 0.0577 - val_mean_absolute_error: 0.0530
Epoch 7/10

In [37]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                patience=2,
                                                mode='min')
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

hypermodel = tuner.hypermodel.build(best_hps)
history = hypermodel.fit(window.train, epochs=best_epoch, validation_data=window.val, 
                         callbacks=[early_stopping, tensorboard_callback])

Epoch 1/5
112/112 [==============================] - 7s 34ms/step - loss: 0.2157 - mean_absolute_error: 0.0718 - val_loss: 0.0607 - val_mean_absolute_error: 0.0531
Epoch 2/5
112/112 [==============================] - 4s 34ms/step - loss: 0.0737 - mean_absolute_error: 0.0672 - val_loss: 0.0681 - val_mean_absolute_error: 0.0611
Epoch 3/5
112/112 [==============================] - 4s 35ms/step - loss: 0.0724 - mean_absolute_error: 0.0664 - val_loss: 0.0610 - val_mean_absolute_error: 0.0557


In [38]:
val_performance['Hypermodel'] = hypermodel.evaluate(window.val)
test_performance['Hypermodel'] = hypermodel.evaluate(window.test, verbose=0)

print('Hypermodel Validation performance: ', val_performance['Hypermodel'])
print('Hypermodel Test performance: ', test_performance['Hypermodel'])

30/30 [==============================] - 0s 4ms/step - loss: 0.0610 - mean_absolute_error: 0.0557
Hypermodel Validation performance:  [0.0610201470553875, 0.05565759167075157]
Hypermodel Test performance:  [0.04758908599615097, 0.04222652688622475]


### Inference

In [104]:
# make predictions

windowTestBatch = window.test.take(1) # given 60 days of info, make prediction: batch of 32
windowTestBatch

<_TakeDataset element_spec=(TensorSpec(shape=(None, 60, 39), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1, 1), dtype=tf.float32, name=None))>

In [105]:
### Predicted Inflow ### 

predictedInflow = hypermodel.predict(windowTestBatch)

1/1 [==============================] - 0s 39ms/step


In [106]:
### Actual Inflow ### 

actualInflow = [] # predicting 30 days ahead: batch of 32

for inputs, labels in windowTestBatch.as_numpy_iterator():
  actualInflow = np.asarray(labels).flatten()

In [107]:
# compare predicted and actual inflow: MAE 

mean_absolute_error(actualInflow, predictedInflow)

0.04128758